Must be on the "parse_parts" branch of wikiextractor repository

Remaining tasks:
* (DONE) Find out why 'afraidness' wasn't captured
* Why is karaff a root in https://api.etymologyexplorer.com/dev/get_trees?ids[]=23683&compression=0
* make it so that existing entry_connections are used, not overwritten, only overwrite etymologies
* (DONE) Then add it so pronunciations and definitions are updated as well (grab this from the spiders?)
* fix issue with identifying words that were hyphenated
* (DONE) Add connections for definitions (should be able to lift this)
* Improve the regexes to provide more accuracy
* Make the deep learning model to catch wrong branch choices

### 0. Intro

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from dgnutils import *
sys.path.append("/Users/nish/development/git/etymology_explorer/modules")
from ety_utils import *
session=requests.session()

08/04/20 dgnutils update loaded!


In [2]:
u = os.environ['RDS_ETY_USER']; p = os.environ['RDS_ETY_PASSWORD']; h = os.environ['RDS_ETY_HOST']
dev_conn, dev_cursor = connect('etymology_explorer_dev', user=u, password=p, host=h)
train_conn, train_cursor = connect('training_data', user=u, password=p, host=h)
# test_conn, test_cursor = connect('etymology_explorer_test', user=u, password=p, host=h)
# stage_conn, stage_cursor = connect('etymology_explorer_staging', user=u, password=p, host=h)
# temp_conn, temp_cursor = connect('temp', user=u, password=p, host=h)

{'etymologies': '707.61 MB',
 'entry_connections': '676.97 MB',
 'entry_definitions': '664.67 MB',
 'entry_pos': '442.06 MB',
 'connection_sources': '269.44 MB',
 'entry_etymologies': '218.72 MB',
 'connections': '156.80 MB',
 'entry_pronunciations': '145.22 MB',
 'languages': '3.00 MB',
 'permanent_errors': '0.03 MB',
 'common_words': '0.03 MB',
 'wiktionary_page_dne': '0.02 MB',
 'transliterations': '0.02 MB',
 'random_etymologies': '0.02 MB',
 'progeny': '0.02 MB',
 'merged_etymologies': '0.02 MB',
 'ancestors': '0.02 MB',
 'kin': '0.02 MB'}

### X. Full Process (1-3)

In [ ]:
# Create test xml
# head -176534 enwiktionary-latest-pages-articles.xml > test.xml
# echo "</mediawiki>" >> test.xml
# wc -l test.xml
# tail -100 test.xml
# ./WikiExtractor.py --processes 4 -o output_test --json --no_templates --lists --sections input/test.xml

#### Class

In [ ]:
log_level('d')
wp = WikiProcessor(
    '/Users/nish/development/git/wikiextractor/', 
    channel='dev', 
    store_intermediates=True,
)
# wp = WikiProcessor('/Users/nish/development/git/wikiextractor/', channel='dev', store_intermediates=True)

In [ ]:
# test - 21s
# dev - 1 hr 45 min
wp.process_wikidump()

In [ ]:
notify('done!')

In [ ]:
# test_conn, test_cursor = connect('etymology_explorer_test', user=u, password=p, host=h)
# test_cursor.e('SELECT * FROM entry_etymologies')
# test_cursor.e('SHOW TABLES')

In [ ]:
# fix dict_insert to allow batches

In [ ]:
database_sizes(wp.cursor) #47s for dev
# test = {'connection_sources': 3860, 'connections': 3344, 'entry_connections': 4008, 'entry_definitions': 7250,
#  'entry_etymologies': 2469, 'entry_pos': 4011, 'entry_pronunciations': 1999, 'etymologies': 5535,'languages': 13154}

# dev = {'connection_sources': 2174854,'connections': 2056605,'entry_connections': 7061624,
#  'entry_definitions': 9053405, 'entry_etymologies': 1447349, 'entry_pos': 6689190, 'entry_pronunciations': 1410408,
#  'etymologies': 7343477,'languages': 13154}

In [ ]:
for w in wp.node_connections:
    print(f"{w[0]['language']:20} {w[0]['word']:20} {w[1]['word']:20} {w[1]['language']:20}")

#### Secondary Databases

In [ ]:
create_transliteration_table(dev_conn, drop_table=False, test=False)

In [ ]:
# Related words or descendant words
# for some words I want to see the related words
    # which words?
# For some words I want to see the descendant words
    # which words? if they have interesting data. Does it depend on the position in tree? Could depend on the
    # number of descendants and the quality

In [ ]:
# Add definitions to etymologies table

#### Individual Functions (Old)

In [ ]:
# len(wp.unmatched_words)
# wp.unmatched_words

In [ ]:
wp.node_connections

In [ ]:
test_ety = wp.cursor.e('SELECT * FROM etymologies')

In [ ]:
test_ety

In [ ]:
test_conn, test_cursor = connect('etymology_explorer_test', user=u, password=p, host=h)
conn = test_conn
output_dir = '/Users/nish/development/git/wikiextractor/output/'
cache_dir = '/Users/nish/development/git/wikiextractor/input/' # None for trio calls

In [ ]:
test = True
log_level('d')
with conn.cursor() as cursor:
    refresh_tables(cursor, ['etymologies', 'languages'], test=True)      
    if test:
        logging.debug(f'Inserting etymologies and languages data due to TEST')
        cursor.e('INSERT INTO etymologies SELECT * FROM etymology_explorer_prod.etymologies LIMIT 100')
        cursor.e('INSERT INTO languages SELECT * FROM etymology_explorer_prod.languages')
    wl_2_id, next_wl_2_id = make_wl_2_id_values(cursor)
    processed_wikidump = load_wikidump_etymologies(output_dir, test=True);
    new_word_lang_pairs, wl_2_id, next_wl_2_id = insert_new_word_lang_pairs(cursor, wl_2_id, next_wl_2_id, processed_wikidump)
    ec_dict, ee_dict, en_dict = create_and_insert_mysql_data(cursor, processed_wikidump, wl_2_id, cache_dir) # no cache
    
    language_dict = create_language_dict(cursor); language_dict['qfa-adm-pro']
    wikitext_part_array = get_wikitext_part_array(new_word_lang_pairs, wl_2_id, ec_dict, ee_dict)
    all_connections, missed_etymologies = get_connections_from_wikitext_parts(wikitext_part_array)
    node_connections = get_nodes_from_connections(all_connections, language_dict)
    roots, descs, table_sources, entry_numbers, unmatched_words = get_mysql_data_from_nodes(cursor, node_connections, en_dict, wl_2_id, next_wl_2_id)
    insert_unmatched_words_into_mysql(cursor, unmatched_words)
    insert_connections_into_mysql(cursor, roots, descs, table_sources, entry_numbers)
conn.commit()
print('Finished')


In [ ]:
# End-to-end Testing: I want to run the entire process with a reduce set of entries, and then it would be easy to change that to include entries that have trouble
# Then I know I wont waste time running the entire processes
# Setup
# test_conn, test_cursor = connect('etymology_explorer_test', user=u, password=p, host=h)
test_conn, test_cursor = connect('etymology_explorer_test', user=u, password=p, host=h)
conn = test_conn
test = True
del_after = False
output_dir = '/Users/nish/development/git/wikiextractor/output/'
cache_dir = '/Users/nish/development/git/wikiextractor/input/'
log_level('i')
with conn.cursor() as cursor:
    
    refresh_tables(cursor, exclude=['etymologies', 'languages'], test=True)      
    if test:
        logging.debug(f'Inserting etymologies and languages data due to TEST')
        cursor.e('INSERT INTO etymologies SELECT * FROM etymology_explorer_prod.etymologies LIMIT 100')
        cursor.e('INSERT INTO languages SELECT * FROM etymology_explorer_prod.languages')

    # IGNORE this step since it can be run and left in place
    # ./WikiExtractor.py --processes 4 -o output --json --no_templates --lists --sections input/enwiktionary-latest-pages-articles.xml
    
#     logging.debug('Creating wl_2_id dictionary...')
#     wl_2_id = {(e['word'], e['language_name']):e['_id'] for e in cursor.d('SELECT * FROM etymologies') }
    wl_2_id, next_wl_2_id = make_wl_2_id_values(cursor)


    # Then take the dump files and make the entry_etymologies (with wikitext) and new words
    processed_wikidump = load_wikidump_etymologies(output_dir, test=True);
    
#     logging.info(f'Found {len(processed_wikidump)} words from the wikiextractor output')

    new_word_lang_pairs, _, _ = insert_new_word_lang_pairs(cursor, wl_2_id, next_wl_2_id, processed_wikidump)

    # Then take the entry_etymologies and process the wikitext so they are clean
    ec_dict, ee_dict, en_dict = create_and_insert_mysql_data(test_cursor, processed_wikidump, wl_2_id, cache_dir)

    # Then make the connections based on the wikitext etymologies
    logging.info(f'Gathering data for connections...')
    
    data = [{
        '_id':wl_2_id[wl],
        'entry_id': ec_dict[wl_2_id[wl]],
        'word':wl[0],
        'language_name':wl[1],
        'wikitext':ee_dict.get(ec_dict[wl_2_id[wl]], None)
    } for wl in new_word_lang_pairs]
    if del_after: 
        del ec_dict
        del en_dict
        del ee_dict
    logging.debug(f'Got {len(list(d for d in data if d["wikitext"]))} with wikitext and {len(list(d for d in data if not d["wikitext"]))} without wikitext')
    
    logging.info(f'Gathering wikitext parts for connections...')
    wikitext_part_array = [get_wikitext_parts_dict(entry) for entry in data if entry['wikitext']]
    if del_after: del data
    
    missed_etymologies = []
    all_connections = []
    for i, wikitext_parts in enumerate(wikitext_part_array):
#         if i % 20000 == 0: print(f'\r{i}/{len(wikitext_part_array)}', end='')
        connections, missed_parts = get_entry_connections(wikitext_parts)

        if missed_parts:
            missed_etymologies.append(missed_parts)
        if connections:
            all_connections += connections
    if del_after: del wikitext_part_array

    logging.debug(f'Gathered {len(all_connections)} connections...')
    
    language_dict = create_language_dict(cursor); language_dict['qfa-adm-pro']
    
    logging.info(f'Converting connections into nodes...')
    nodeConnections = []
    for i, connection in enumerate(all_connections):
#         if i % 50000 == 0: print(f'\r{i}/{len(all_connections)}', end='')
        nodeConnections += getNodeConnections(connection, language_dict)
#     nodeConnections[2003]
    if del_after: del all_connections
    
    logging.info(f'Converting nodes into mysql friendly data and adding missed words...')
    unmatched_words = []
    next_wl_2_id = max(wl_2_id.values())+1
    
    # Define local function
    def getOrCreateIdWithDict(word, lang, cursor):
        global next_wl_2_id, unmatched_words
        word = clean_word(word) # removes asterisk and other issues
        try:
            _id = wl_2_id[(word, lang)]
        except KeyError:
            word = remove_diacritics(word)
            try:
                _id = wl_2_id[(word, lang)]
            except KeyError:
                _id = next_wl_2_id
                wl_2_id[(word, lang)] = next_wl_2_id
                next_wl_2_id += 1
                unmatched_words += [[_id, word, lang]]
        return _id
    
    roots = []; descs = []; table_sources = []; entry_numbers = [];
    for i, (desc, root, entry_id) in enumerate(nodeConnections):
        desc_id = getOrCreateIdWithDict(desc['word'], desc['language'], cursor)
        root_id = getOrCreateIdWithDict(root['word'], root['language'], cursor)

        roots.append(root_id)
        descs.append(desc_id)
        table_sources.append(entry_id)
        entry_numbers.append(en_dict[entry_id])
    if del_after:
        del en_dict
        del nodeConnections
    
    logging.info(f'Found {len(unmatched_words)} unmatched words. Inserting...')
    _ids,words,langs = zip(*unmatched_words)
    value_dict = {'_id':_ids, 'word':words,'language_name':langs}
    insert(
        cursor, 
        'etymologies', 
        many=True,
        **value_dict
    )

    # Then put those connections in the database
    
    logging.info(f'Found {len(roots)} connection_sources. Inserting...')
    # Connection sources 1.5min
    insert(cursor, 'connection_sources', many=True, 
        **{'root':roots,'descendant':descs,'table_source':table_sources,'entry_number':entry_numbers})

    # Connection data 1 min
    # make a set for roots, desc
    conn_set = set(zip(roots,descs))
    roots_set = [s[0] for s in conn_set]
    descs_set = [s[1] for s in conn_set]
    logging.info(f'Found {len(roots_set)} connection_sources. Inserting...')
    insert(cursor,'connections',ignore=True,many=True, **{'root':roots_set,'descendant':descs_set,})
    
conn.commit()
    # Add extra connections based on definitions and such

    # Then remove any messed up items (try to move these actions upstream)

    # make secondary databases

In [ ]:
database_sizes(test_cursor)
# connection_sources 3227
# connections 2878
# entry_connections 4008
# entry_definitions 7250
# entry_etymologies 2469
# entry_pos 4011
# entry_pronunciations 1999
# etymologies 5345
# languages 13154

In [ ]:
# nodeConnections[0]
# roots[:5]
# getOrCreateIdWithDict('man', 'Old Frisian', test_cursor)
# dev_conn, dev_cursor = connect('etymology_explorer_dev', user=u, password=p, host=h)
# test_conn, test_cursor = connect('etymology_explorer_test', user=u, password=p, host=h)
# create_language_dict(test_cursor)['ofs']
# prod_conn, prod_cursor = connect('etymology_explorer_prod', user=u, password=p, host=h)
# language_dict = create_language_dict(dev_cursor); language_dict['qfa-adm-pro'], language_dict['ofs']
# language_dict = create_language_dict(test_cursor); language_dict['qfa-adm-pro'], language_dict['ofs']
# language_dict = create_language_dict(prod_cursor); language_dict['qfa-adm-pro'], language_dict['ofs']


### 1. Wiktionary Dump => Mysql (entry_etymologies)

In [ ]:
dev_conn, dev_cursor = connect('etymology_explorer_dev', user=u, password=p, host=h)

#### Wiktionary Dump => etymologies (dict)

In [ ]:
# Run this to get the dump processed
# ./WikiExtractor.py --processes 4 -o output --json --no_templates --lists --sections input/enwiktionary-latest-pages-articles.xml

In [ ]:
output_dir = '/Users/nish/development/git/wikiextractor/output/'
# etymologies = loadWikidumpEtymologies(output_dir)
test_etymologies = loadWikidumpEtymologies(output_dir, test=True);

In [ ]:
len([v_i['etymology'] for l in etymologies.values() for v in l.values() for v_i in v if 'etymology' in v_i])

Add the new word-language pairs into etymologies table (use sets)

In [ ]:
current_etymologies = set(dev_cursor.e('SELECT word,language_name FROM etymologies')); len(current_etymologies) #2m

In [ ]:
new_etymologies = [(word, lang) for word,word_v in test_etymologies.items() for lang in word_v.keys()]
len(new_etymologies), new_etymologies[:5]

In [ ]:
max_id = dev_cursor.e('SELECT MAX(_id) FROM etymologies')[0][0]; max_id

In [ ]:
new_new_etymologies = new_etymologies - current_etymologies; len(new_new_etymologies)

In [ ]:
del new_etymologies
del current_etymologies

In [ ]:
values = []
for i,n in enumerate(new_new_etymologies):
    if not i % 50000: print(f'\r{i}/{len(new_new_etymologies)}', end='')
    values.append({'_id': i+max_id+1, 'word':n[0], 'language_name':n[1], 'common_descendant':''})

In [ ]:
# 9minutes
dev_cursor.dict_insert(values, 'etymologies')

In [ ]:
dev_conn.commit()

#### Gather table values from etymologies dict

Create the entry_connections and entry_etymologies tables based on the word-languages and etymologies

In [ ]:
wl_2_id = {(e['word'], e['language_name']):e['_id'] for e in dev_cursor.d('SELECT * FROM etymologies') }
last_id = max(wl_2_id.values())

In [ ]:
# Get a short sample
{k:etymologies[k] for k in list(etymologies)[:2]}

In [ ]:
temp = create_mysql_data_from_processed_wikiextraction_data(test_etymologies, wl_2_id, log=False)
entry_connections_dict_list, \
entry_etymologies_dict_list, \
entry_pos_dict_list, \
entry_pronunciations_dict_list, \
entry_definitions_dict_list, \
etymologies_dict_list = temp
del temp

In [ ]:
[len(w) for w in entry_connections_dict_list.values()]

Remove wikitext

In [ ]:
log_level('i')

In [ ]:
cache_dir = '/Users/nish/development/git/wikiextractor/input/'

In [ ]:
entry_pronunciations_dict_list['pronunciation'] = multiParseWikitextSentences(
    entry_pronunciations_dict_list['pronunciation'], 
    cache_file=cache_dir+'pron.wik'
)

In [ ]:
entry_etymologies_dict_list['etymology'] = multiParseWikitextSentences(
    entry_etymologies_dict_list['wikitext'],
    cache_file=cache_dir+'ety.wik'
)

In [ ]:
entry_definitions_dict_list['definition'] = multiParseWikitextSentences(
    entry_definitions_dict_list['definition'],
    cache_file = cache_dir + 'def.wik'
)

In [ ]:
# List of templates used in pronunciations
set((splitParts(e[e_i[0]+2:e_i[1]-2])[0], len(splitParts(e[e_i[0]+2:e_i[1]-2]))) for e in entry_pronunciations_dict_list['pronunciation'] for e_i in findMatchingBraces(e))

In [ ]:
[[len(v) for v in e.values()] for e in [entry_connections_dict_list, entry_etymologies_dict_list, entry_pos_dict_list, entry_pronunciations_dict_list, entry_definitions_dict_list, etymologies_dict_list]]

In [ ]:
# MAKE THE normal text from wiki text here for entry_etymologies

#### Insert the table values into mysql

In [ ]:
defs = dev_cursor.d('SELECT * FROM entry_definitions'); len(defs), defs[:5]

In [ ]:
[d['definition'] for d in defs[:5]]

In [ ]:
cache_dir = '/Users/nish/development/git/wikiextractor/input/'
fixed_defs = multiParseWikitextSentences(
    [d['definition'] for d in defs],
    cache_file = cache_dir + 'def.wik'
)

In [ ]:
multiParseWikitextSentences(sentences)

In [ ]:
i=100000
entry_etymologies_dict_list['etymology'][i:i+100]
# entry_etymologies_dict_list['entry_id'][i:i+100]
entry_etymologies_dict_list['wikitext'][i:i+100]
# entry_etymologies_dict_list['entry_id'][i:i+100]
# entry_connections_dict_list['etymology_id'][100002]

In [ ]:
len(entry_connections_dict_list['etymology_id']), len(entry_connections_dict_list['entry_id']), len(entry_connections_dict_list['entry_number']), len(entry_etymologies_dict_list['entry_id']), len(entry_etymologies_dict_list['etymology'])

In [ ]:
dev_cursor.e('SHOW CREATE TABLE etymologies')

In [ ]:
# dev_cursor.d('SELECT * FROM entry_connections LIMIT 5')
# dev_cursor.d('SELECT COuNT(*) FROM entry_etymologies LIMIT 5')
# dev_cursor.d('DESCRIBE entry_etymologies')
# dev_cursor.d('DELETE FROM entry_connections'); print('done')
# dev_cursor.d('DELETE FROM entry_etymologies'); print('done')
# dev_cursor.d('DELETE FROM entry_pos'); print('done')
# dev_cursor.d('DELETE FROM entry_definitions'); print('done')
# dev_cursor.d('DELETE FROM entry_pronunciations'); print('done')
dev_cursor.e('DROP TABLE IF EXISTS entry_etymologies ')
dev_cursor.e('CREATE TABLE `entry_etymologies` (`entry_id` int(11) NOT NULL, `wikitext` text COLLATE utf8mb4_bin, `etymology` text COLLATE utf8mb4_bin, PRIMARY KEY (`entry_id`)) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_bin')
# dev_conn.commit()
# dev_cursor.d('DESCRIBE entry_etymologies')


In [ ]:
dev_cursor.e('SELECT * FROM entry_connections LIMIT 1')

In [ ]:
insert(dev_cursor, 'etymologies', batch_size=50000, **etymologies_dict_list)
# del etymologies_dict_list

In [ ]:
entry_etymologies['etymology'] = [None]*1447349

In [ ]:
[len(e) for e in entry_etymologies_dict_list.values()]

In [ ]:
insert(dev_cursor, 'entry_etymologies', many=True, batch_size=50000, **entry_etymologies_dict_list)
# del entry_etymologies_dict_list

In [ ]:
insert(dev_cursor, 'entry_connections', many=True, batch_size=50000, **entry_connections_dict_list)
del entry_connections_dict_list

In [ ]:
insert(dev_cursor, 'entry_pos', many=True, batch_size=50000, **entry_pos_dict_list)
del entry_pos_dict_list

In [ ]:
insert(dev_cursor, 'entry_definitions', many=True, batch_size=50000, **entry_definitions_dict_list)
# del entry_definitions_dict_list

In [ ]:
insert(dev_cursor, 'entry_pronunciations', many=True, batch_size=50000, **entry_pronunciations_dict_list)
del entry_pronunciations_dict_list

In [ ]:
dev_conn.commit()

In [ ]:
# dev_cursor.e('SELECT COUNT(*) FROM entry_etymologies')
dev_cursor.e('SELECT * FROM entry_etymologies LIMIT 1')

In [ ]:
entry_etymologies_dict_list['etymology'] = multiParseWikitextSentences(entry_etymologies_dict_list['wikitext'])

In [ ]:
parseTemplates('From {{inh|en|enm|cat}}, {{m|enm|catte}}, from {{inh|en|ang|catt||male cat}}, {{m|ang|catte||female cat}}, from {{inh|en|gem-pro|*kattuz}}.\nThe Germanic word is generally thought to be from {{der|en|LL.|cattus||domestic cat}} (c. 350, Palladius), from {{der|en|la|catta}} (c. 75 {{small|A.D.}}, Martial), from an {{der|en|afa}} language. This would roughly match how domestic cats themselves spread, as genetic studies suggest they began to spread out of the Near East / Fertile Crescent during the Neolithic (being in Cyprus by 9500 years ago, and Greece and Italy by 2500 years ago), especially after they became popular in Egypt. However, every proposed source word has presented problems. Adolphe Pictet and many subsequent sources refer to Barabra (Nubian) {{m|onw|tr=kaddîska}} and "Nouba" (Nobiin) {{m|fia|kadīs}} as possible sources or cognates, but M. Lionel Bender says the Nubian word is a loan from {{noncog|ar|قِطَّة}}. Jean-Paul Savignac suggests the Latin word is from an Egyptian precursor of {{cog|cop|ϣⲁⲩ||tomcat}} suffixed with feminine {{m|egy|-t}}, but John Huehnergard says "the source [...] was clearly not Egyptian itself, where no analogous form is attested."\nHuehnergard opines it is "equally likely that the forms might derive from an ancient Germanic word, imported into Latin and thence to Greek and to Syriac and Arabic". Guus Kroonen also considers the word to be native to Germanic (due to morphological alternations) and Northern Europe, and suggests that it might ultimately be borrowed from Uralic, compare {{noncog|se|gađfe||female stoat}} and {{noncog|hu|hölgy||stoat; lady, bride}} from {{noncog|urj-pro|*käďwä||female (of a fur animal)}}.\nRelated to {{cog|sco|cat}}, {{cog|fy|kat}}, {{cog|frr|kåt}} and {{m|frr|kaat}}, {{cog|nl|kat}}, {{cog|da|kat}}, {{cog|no|katt}}, {{cog|sv|katt}}, {{cog|nds-de|Katt}} and {{m|nds-de|Katte}}, {{cog|de|Katze}}, {{cog|gsw|Chatz}}, {{cog|is|köttur}}, {{cog|af|kat}}, {{cog|la|cattus}}, {{cog|fr|chat}}, {{cog|nrf|cat}}, {{cog|oc|cat}}, {{cog|pt|gato}}, {{cog|es|gato}}, {{cog|rup|cãtush}}, {{cog|gd|cat}}, {{cog|ga|cat}}, {{cog|br|kazh}}, {{cog|cy|cath}}, {{cog|kw|kath}}, as well as {{cog|grc|κάττα}}, {{cog|el|γάτα}}, and from the same ultimate source {{cog|ru|кот}}, {{cog|uk|кіт}}, {{cog|be|кот}}, {{cog|pl|kot}}, {{cog|csb|kòt}}, {{cog|lt|katė}}, and more distantly {{cog|hy|կատու}}, {{cog|eu|katu}}, {{cog|he|חתול|tr=khatúl}}, {{cog|ar|قِطَّة}} alongside dialectal Maghrebi Arabic {{m|ar|قَطُّوس}} (from Berber, probably from Latin).', session)

### 2. Mysql(entry_etymologies) => connections array

#### Text Management

In [ ]:
a = 'Cf. French'
remove_starting_clauses(a)
# remove_matching_parens(a)
# remove_lone_languages(a)
preprocess_etymology(a)

In [ ]:
etymology = 'BULLET::::-Cf. banana'
etymology = preprocess_etymology(etymology); etymology
etymology = etymology.replace(u'\xa0', u' ').replace('\n', ' '); etymology
etymology = replace_bullets(etymology); etymology
etymology = remove_matching_parens(etymology); etymology
etymology = remove_lone_wikis(etymology); etymology
etymology = replace_cf(etymology); etymology
etymology = remove_starting_clauses(etymology); etymology

In [ ]:
# Not doing this yet
# RE_QUOTES = r"\"(\w+)\"(?![^{]*})"
# test = 'from {{"test"}} test {{test|test}} "in"'
# re.sub(RE_QUOTES, "{{eeQuote|\g<1>}}", test)

#### Processing Functions

In [ ]:
test = {'entry_id': 496,'etymology': 'Probably {{m|mkh-okm|hvat}}, from {{m|mkh-okm|hvat}}','entry_number': 3,'word': 'วัด','language_name': 'Thai'}
assert get_wikitext_parts_dict(test) == {'{{eeStart|Thai|วัด}}': {'entry_id': 496,'preceding_text': '','wikitype': 'eeStart','place': 0,'following_text': ' : Probably ','following_wikitext_array': ['{{m|mkh-okm|hvat}}']},'{{m|mkh-okm|hvat}}': {'entry_id': 496,'preceding_text': ' : Probably ','wikitype': 'm','place': 1,'preceding_wikitext_array': ['{{eeStart|Thai|วัด}}'],'following_text': ', from ','following_wikitext_array': ['{{_m|mkh-okm|hvat}}']},'{{_m|mkh-okm|hvat}}': {'entry_id': 496,'preceding_text': ', from ','wikitype': 'm','place': 2,'preceding_wikitext_array': ['{{m|mkh-okm|hvat}}']}}
assert get_entry_connections(get_wikitext_parts_dict(test)) == ([['{{eeStart|Thai|วัด}}', '{{m|mkh-okm|hvat}}', 'm', 496],['{{m|mkh-okm|hvat}}', '{{m|mkh-okm|hvat}}', 'm', 496]],[])

In [ ]:
get_entry_connections(get_wikitext_parts_dict(test))

#### Evaluation

##### Data gather

In [ ]:
# 140s
sql_stmt = """
SELECT * FROM entry_etymologies ee
INNER JOIN entry_connections ec ON ec.entry_id=ee.entry_id
INNER JOIN etymologies e on e._id=ec.etymology_id
"""
data = dev_cursor.d(sql_stmt); len(data)

In [ ]:
# 90s 
wikitext_part_array = [get_wikitext_parts_dict(entry) for entry in data if entry['wikitext']]
list(wikitext_part_array[0].items())[0]

In [ ]:
# 36.7 seconds due to json
missed_etymologies = []
all_connections = []
for i, wikitext_parts in enumerate(wikitext_part_array):
    if i % 20000 == 0: print(f'\r{i}/{len(wikitext_part_array)}', end='')
    connections, missed_parts = get_entry_connections(wikitext_parts)
    
    if missed_parts:
        missed_etymologies.append(missed_parts)
    if connections:
        all_connections += connections

In [ ]:
len(all_connections), len(missed_etymologies)
# missing amount
# 99075: update 5, before adding the {{etyl}} bypass
# 56616: update 6, after adding the {{etyl}} bypass
# 54367: update 7, after adding the removal of ".", ";", ":" phrases in opening lines
# 55381: update 8, after early replacing of {{etyl}} and {{inh|-}}
# 48623: update 9, after updating cognate text list
# 47586: update 10, after updating the single branch text list
# 52406: update 11, after updating the order of the etymology text regexes
# 47360 : update 11, after fixing lone language regex
# 34674 : update 12, after adding a cognates regex
# 33729 : update 13, after updating the single branch regex
# 37413 : update 14, after fixing the single branch regex to exclude post key word dashes
# 35277 : update 15, added several single path etymology items
# 34293 : update 16, added some replacement words
# 23360 : update 17, flexible cognate if the text is cognate for sure
# 31430 : update 18, added replacements for Cf. and Bullet::::-
# 21809 : update 19, Many miscellaneous updates
# 21632 : update 20, Fixed duplicated wikitext_part entries 

##### Eval

In [ ]:
# Look at the sentences that are created from the REGEX
temp1 = [list(l.items())[0][1].get('following_text','') for l in wikitext_part_array]; temp1[0]
temp1_matches = [t for t in temp1 if re.search(RE_FROM, t)]; temp1_matches
Counter(temp1_matches).most_common()

In [ ]:
' : According to Beekes, of '
' : From the same source as '
' : of '
' : First recorded as a given name of Latvians in the end of the 19th century. From ' # period ok?
' : Modified from above. Compare '
' : According to Beekes, from the same '
' : Forms of '
' : From a poem by '
' : From the book "'
' : Possibly of '
' : Equivalent of '
' : The kanji spelling is an example of '
' : From Magadhi Prakrit, from Old-Indo-Aryan, akin to '
' : From the "műv-" stem of '
' : Usually considered a word of ' (8)
' : Related to or derived from '
' : One of the variant spellings of Nichols, from Nichol, a '
' : From the name of the local tribe, the '
' : From "*teda", from "*tendan", from '
' : Taboo of mentioning '
' : From its use in "'
' : Use of '
' : From Yueyang Lou Ji by '
' : Derived from the name "'
' : From "-er-" + '
' : From From '
' : From class 8 '
' : From unknown '
Stopped at 5

In [ ]:
find_text = ' : From the '
temp2 = [list(l.items())[0] for l in wikitext_part_array]; temp2[0]
temp2_match = [t for t in temp2 if t[1].get('following_text','')==find_text]; temp2_match[:20]

In [ ]:
pdb.pm()

In [ ]:
len(temp)

In [ ]:
temp = [d for d in data if re.search(r'^(\w+? )+?to {', d['etymology'])]; temp[2]

In [ ]:
preprocess_etymology('From the Interlingua-English Dictionary.\nFrom {{der|ia|en|plantation}}, from')

In [ ]:
for i in range(100,200):
    me = missed_etymologies[i][0]; me
    if me['entry_id'] <= 5214: continue
        
        
    d = data[me['entry_id']]; d
#     d = data[502]; d
    de = d['etymology']; de
    # print(me['entry_id'], de)
    # log_level('d')
    # wtpd = get_wikitext_parts_dict(d); wtpd
    # cons = get_entry_connections(wtpd); cons
    # log_level('w')
    # cons
    print(de, f"{d['word']}#{d['language_name']}", d['entry_id'])
    input('continue?')
    clear_output()

In [ ]:
from_str('From the Roman name, ')

In [ ]:
s = 'Attested in {{inh|fr|fro|-}} as {{m|fro|bien tost}}.'
bool(re.search(RE_FROM, ' : ' + s))
remove_lone_languages('asdf {{der|ja|ltc|sort=けん|-}} asdf')

In [ ]:
# Begins with a language # 
Arabic {{l|ar|نَسْخ|gloss=transcription}} #  نستعليق Arabic 4956

# Ensure this doesn't happen
Of {{l|en|Istrian}} origin. lima Italian 4874

# Acceptable wikipedia pages ('named after...', etc) with {{w|}} at the end
Named after the 18th-century Swiss mathematician {{w|Leonhard Euler}} # Euler's totient function English 4663

# Cognate
The original meaning was "row," "rank," later "partition," possibly related to {{
The current spelling was adopted in the 1690s to emulate the spelling but not the pronunciation (at least originally) of the equivalent modern {{cog|fr|quai}}
Although formally a perfectly regular form synchronically, it has most likely become to be used as a greeting because of a paronymic contamination with earlier {{cog|la|havĕ}} 
Coined by G. de la Landelle together with {{m|fr|aviateur}}; from {{suffix|fr|avis|t1=bird|lang1=la|ation}}. aviation French 2931

# single path
Originated 1595–1605 from {{etyl|LL.|en}} {{m|la|cooperatus||work with}}
Deliberate typographical error for {{m|en|typo}}

# Improve regex (add to testing) or from strings
'Recorded since c.1330 as {{inh|en|enm|pronouncen||to utter, declare officially}}' pronounce English 554
'Coined based on {{etyl|la|en}}'
                
# Uncertain
'Translating {{der|la|grc|Βεελζεβούλ}} and {{der|la|he|בעל זבוב||fly-lord|tr=ba‘al-z'būb}}; perhaps a corruption of Beelzebul, meaning "Lord of the High Place", with "-bul" altered to "-bub" to change the meaning to "Lord of the Flies". Beelzebub Latin 953'
Genova Italian 125
'A reduplication of {{ja-r|もし}}'
'1922, {{clipping|en|aborigine|nocap=1}}'
'Old Swedish "rote",'
From {{der|ja|ltc|sort=けん|-}} {{ltc-l|兼}}. 兼 Japanese 943
Related to {{m|fi|kostua}}, {{cog|izh|kossia}} and {{cog|vot|kõssõa}}. Sense contaminated by the family of {{m|fi|kastaa}}; see {{m|fi|kostua}} for more. kostea Finnish 1043
Alteration by association with {{m|en|-ary}} of the {{der|en|fr|axillaire||of the axilla}} axillary English 2831
From "i"-stem + {{m|la|-cus}}-icus Latin 20

# Definitely ignore
{{la-IPA|ēvolve}}
Coined by {{w|H.G. Wells}}

In [ ]:
de

In [ ]:
i = 304455
# me = missed_etymologies[i][0]; me
# d = data[me['entry_id']]; d
d = data[i]; d
de = d['etymology']; de
# print(me['entry_id'], de)
log_level('d')
wtpd = get_wikitext_parts_dict(d); wtpd
print(wtpd)
cons = get_entry_connections(wtpd); cons
log_level('w')
cons


In [ ]:
elif wikitype in COGNATE_TYPES \
                or (cognate_str(preceding_text) and wikitype in SINGLE_PATH_TYPES):

In [ ]:
etymology1

In [ ]:
cons

In [ ]:
# Analysis

In [ ]:
parseTemplates('{{af|eo|geometrio|t1=geometry|-a|pos2=adjectival suffix}}')

In [ ]:
[v for m in missed_etymologies for v in m if '{{confix|' in v['wikitext']]

In [ ]:
preceding_texts = [v['preceding_text'] for m in missed_etymologies for v in m]; len(preceding_texts)

In [ ]:
wiki_text_errors = [v['wikitext'] for m in missed_etymologies for v in m if v['preceding_text']==' : From ']; len(wiki_text_errors)
wiki_text_type_errors = [splitParts(w[2:-2])[0] for w in wiki_text_errors]

In [ ]:
# most_common = Counter(wiki_text_type_errors).most_common(); most_common
most_common = Counter(preceding_texts).most_common(); most_common

In [ ]:
print(f'wikitext_part_array: {len(wikitext_part_array)}')
print(f'completed_etymologies: {len(wikitext_part_array) - len(missed_etymologies)}')
print(f'missed_etymologies:  {len(missed_etymologies)}')
print(f'all_connections: {len([a for a in all_connections if a])}')
print(f'missed_parts:  {len(missed_parts)}')


print(all_connections[199999])

In [ ]:
entry['etymology']

In [ ]:
print(json.dumps(wikitext_parts, indent=2))

In [ ]:

#         print(s,e)
#         res += wikitext[cur:s] + self.expandTemplate(wikitext[s + 2:e - 2])
#     cur = e
#     leftover
#     res += wikitext[cur:]
    # logging.debug('%*sexpand> %s', self.frame.depth, '', res)
#     return res


In [ ]:
# Testing

In [ ]:
t1

In [ ]:
get_entry_connections(t2)

In [ ]:
t0 = {
    '{{eeStart}}': {'preceding_text':'','wikitype':'eeStart','place': 0,'following_text':' : From ','following_wikitext_array': ['{{inh|1}}']},
    '{{inh|1}}': {'entry_id': 0,'preceding_text': ' : From ','wikitype': 'inh','place': 1,'preceding_wikitext_array': ['{{eeStart}}'],'following_text': ', from ','following_wikitext_array': ['{{inh|2}}']},
    '{{inh|2}}': {'entry_id': 0,'preceding_text': ', from ','wikitype': 'inh','place': 2,'preceding_wikitext_array': ['{{inh|1}}'],'following_text': '. Germanic cognates include ','following_wikitext_array': ['{{cog|1}}']},
    '{{cog|1}}': {'entry_id': 0,'preceding_text': '. Germanic cognates include ','wikitype': 'cog','place': 3,'preceding_wikitext_array': ['{{inh|2}}'],'following_text': ' (','following_wikitext_array': ['{{cog|2}}']},
}
assert get_entry_connections(t0) == ([['{{eeStart}}', '{{inh|1}}', 'inh'],['{{inh|1}}','{{inh|2}}','inh'],['{{eeStart}}', '{{cog|1}}', 'cog']], [])
t1 = {'{{eeStart}}': {'preceding_text': '','wikitype': 'eeStart','following_text': ' : From ','following_wikitext_array': ['{{inh|1}}']},
     '{{inh|1}}': {'preceding_text': ' : From ','wikitype': 'inh','preceding_wikitext_array': ['{{eeStart}}'],'following_text': ', ','following_wikitext_array': ['{{m|1}}']},
     '{{m|1}}': {'preceding_text':', ','wikitype':'m','preceding_wikitext_array': ['{{inh|1}}'],'following_text': ', borrowed from ','following_wikitext_array': ['{{bor|1}}']},
     '{{bor|1}}': {'preceding_text': ', borrowed from ','wikitype': 'bor','preceding_wikitext_array': ['{{m|1}}'],'following_text': ', noun of action from perfect passive participle ','following_wikitext_array': ['{{m|2}}']},
     '{{m|2}}': {'preceding_text': ', noun of action from perfect passive participle ','wikitype': 'm','preceding_wikitext_array': ['{{bor|1}}'],'following_text': ', from verb ','following_wikitext_array': ['{{m|3}}']},
     '{{m|3}}': {'preceding_text': ', from verb ','wikitype': 'm','preceding_wikitext_array': ['{{m|2}}'],'following_text': ', + noun of action suffix ','following_wikitext_array': ['{{m|4}}']},
     '{{m|4}}': {'preceding_text': ', + noun of action suffix ','wikitype': 'm','preceding_wikitext_array': ['{{m|3}}']}}
assert get_entry_connections(t1) == ([['{{eeStart}}', '{{inh|1}}', 'inh'],['{{eeStart}}', '{{m|1}}', 'm'],['{{m|1}}', '{{bor|1}}', 'bor'],['{{inh|1}}', '{{bor|1}}', 'bor'],['{{bor|1}}', '{{m|2}}', 'm'],['{{m|2}}', '{{m|3}}', 'm'],['{{m|2}}', '{{m|4}}', 'm']],[])
t2 = {'{{eeStart}}': {'preceding_text': '','wikitype': 'eeStart','following_text': ' : From ','following_wikitext_array': ['{{inh|en|enm|techere}}']}, 
      '{{inh|en|enm|techere}}': {'preceding_text': ' : From ',  'wikitype': 'inh','preceding_wikitext_array': ['{{eeStart}}'],'following_text': ', equivalent to ','following_wikitext_array': ['{{suffix|en|teach|er}}']},
      '{{suffix|en|teach|er}}': {'preceding_text': ', equivalent to ','wikitype': 'suffix','preceding_wikitext_array': ['{{inh|en|enm|techere}}'],'following_text': '. More at ','following_wikitext_array': ['{{l|en|teach}}']},
      '{{l|en|teach}}': {'preceding_text': '. More at ','wikitype': 'l','preceding_wikitext_array': ['{{suffix|en|teach|er}}']}}
assert get_entry_connections(t2) == ([['{{eeStart}}', '{{inh|en|enm|techere}}', 'inh'],['{{eeStart}}', '{{suffix|en|teach|er}}', 'suffix'],['{{eeStart}}', '{{l|en|teach}}', 'cog']],[])
t3 = {'{{eeStart|English|trink}}': {'entry_id': 2296,  'preceding_text': '',  'wikitype': 'eeStart',  'place': 0,  'following_text': ' : From ',  'following_wikitext_array': ['{{inh|en|enm|treinekys}}']}, '{{inh|en|enm|treinekys}}': {'entry_id': 2296,  'preceding_text': ' : From ',  'wikitype': 'inh',  'place': 1,  'preceding_wikitext_array': ['{{eeStart|English|trink}}'],  'following_text': ' or ',  'following_wikitext_array': ['{{m|enm|trynk}}']}, '{{m|enm|trynk}}': {'entry_id': 2296,  'preceding_text': ' or ',  'wikitype': 'm',  'place': 2,  'preceding_wikitext_array': ['{{inh|en|enm|treinekys}}'],  'following_text': ', but earlier origin is ',  'following_wikitext_array': ['{{unk|en|nocap=1}}']}, '{{unk|en|nocap=1}}': {'entry_id': 2296,  'preceding_text': ', but earlier origin is ',  'wikitype': 'unk',  'place': 3,  'preceding_wikitext_array': ['{{m|enm|trynk}}'],  'following_text': '. Attested in ',  'following_wikitext_array': ['{{w|Anglo-Norman}}']}, '{{w|Anglo-Norman}}': {'entry_id': 2296,  'preceding_text': '. Attested in ',  'wikitype': 'w',  'place': 4,  'preceding_wikitext_array': ['{{unk|en|nocap=1}}']}}
assert get_entry_connections(t3)[0] == [['{{eeStart|English|trink}}', '{{inh|en|enm|treinekys}}', 'inh'],['{{eeStart|English|trink}}', '{{m|enm|trynk}}', 'm']]

In [ ]:
log_level('w')

In [ ]:
log_level('d')
get_entry_connections(t3)

In [ ]:
get_entry_connections(t2)

In [ ]:
test = [d for d in data if '}, {' in d['etymology']]

In [ ]:
e1 = test[100];e1


In [ ]:
e1['etymology']

In [ ]:
log_level('w')

In [ ]:
te1 = get_wikitext_parts_dict(e1); te1
get_entry_connections(te1)

In [ ]:
get_entry_connections(t)

### 3. connections array => Mysql(connections) 

In [ ]:
dev_conn, dev_cursor = connect('etymology_explorer_dev', user=u, password=p, host=h)

#### Functions

Put the connection in the function so that it calls for the language_dict? Make it an optional input for the 
get Node connections since it might not need it

In [ ]:
# One idea is to put this function inside the place I use it to get the contexts right
def getOrCreateIdWithDict(word:str, lang:str, cursor, debug_str=None):
	"""
	Parameters
	==========
	word (str): the word to check. This fn will clean and transliterate, so no need to before
	lang (str): the language to check

	Global Parameters
	=================
	wl_2_id (dict): The dictionary of { (word, lang): id, ...}
	next_wl_2_id (int): The first unused _id in wl_2_id

	Get the _id of a word,language combo from dict of table etymlogies
	If it doesn't exist create a new id based on the max value
	"""
	# Manage the global variables
	global wl_2_id
	global next_wl_2_id 
	global unmatched_words

	try:
		unmatched_words
	except NameError:
		unmatched_words = []

	try:
		wl_2_id
		next_wl_2_id # Assume this is created if wl_2_id is
	except NameError:
		print('Creating wl_2_id, may take ~1 minute')
		wl_2_id, next_wl_2_id = make_wl_2_id_values(cursor)

	# Handle the actual function
	word = clean_word(word) # removes asterisk and other issues

	try:	
		_id = wl_2_id[(word, lang)]
	except KeyError:
		word = remove_diacritics(word)
		try:
			_id = wl_2_id[(word, lang)]
		except KeyError:
			_id = next_wl_2_id
			wl_2_id[(word, lang)] = next_wl_2_id
			if debug_str:
				logging.debug(debug_str)
			next_wl_2_id += 1
			unmatched_words += [[_id, word, lang]]

	return _id


In [ ]:
getConnectionsForID(dev_cursor, 0)

In [ ]:
wl_2_id[('acrocarpous','English')]

In [ ]:
# dev_cursor.e('SELECT * FROM etymologies WHERE _id=1')
# dev_cursor.d('SELECT * FROM entry_connections WHERE etymology_id=1 ')
dev_cursor.d('SELECT * FROM entry_etymologies WHERE entry_id=225005')

In [ ]:
# values = [[n['word'], n['language']] for conn in nodeConnections for n in conn if type(n) != int]; values
# cursor.d('SELECT word, language_name, _id from etymologies WHERE word=%s AND language_name=%s', many=True, values=[[n['word'], n['language']] for conn in nodeConnections for n in conn if type(n) != int])

#### Continue

In [ ]:
language_dict = create_language_dict(dev_cursor); language_dict['qfa-adm-pro']

In [ ]:
nodeConnections = []
for i, connection in enumerate(all_connections):
    if i % 50000 == 0: print(f'\r{i}/{len(all_connections)}', end='')
    nodeConnections += getNodeConnections(connection, dev_cursor)
nodeConnections[2003]

In [ ]:
# nodeConnections[2003]
# getIndexForWordAndLanguage('μόνος', 'Ancient Greek', data)
# getConnectionsForIndex(295123, dev_cursor, data)
# [d for d in data if d['word'] == 'conniption']

In [ ]:
# Checking the word and languages versus expand/parse Templates
# for i, (desc, root, conn_type, index) in enumerate(all_connections):
#     if i <= 4061: continue
#     template = root
        
#     typen, *parts = splitParts(template[2:-2])
    
#     if typen == 'eeStart': continue
#     if typen == 'cog': continue
    
#     try:
#         myResult = getNodesFromTemplate(template, 'root', language_dict)
#     except EmptyWordOrLanguageError as e:
#         print(f'\rEMPTY WORD on {i}', root, end='\n')
#         continue
        
#     if typen in ['m', 'l']:
#         o1 = f"{myResult[0]['word']}"
#     elif typen in ['suffix', 'suf']:
#         o1 = f'{myResult[0]["word"]} + -{myResult[1]["word"]}'
#     elif typen in ['prefix', 'pre']:
#         o1 = f'{myResult[0]["word"]}- + {myResult[1]["word"]}'
#     elif typen in ['affix', 'af', 'compound', 'com']:
#         o1 = ' + '.join(r['word'] for r in myResult)
#     elif typen in ['ja-r']:
#         o1 = f'{myResult[0]["word"]}'
#     else:
#         o1 = f"{myResult[0]['language']} {myResult[0]['word']}"
#     o2 = remove_matching_parens(parseTemplates(template, session=session)) # remove parens to match
        
#     if o1 != o2.replace('\u200e', ''):
#         if typen == 'inh' and parts[2] == myResult[0]['word']: continue
#         print(f'\rERROR on {i}', o1, o2, o1 == o2, root, end='\n')
#     else:
#         print(f'\r{i}/{len(all_connections)}', end='')
#         pass
# #         print(f'{i} ok!', o1, o2)

In [ ]:
assert getNodesFromTemplate('{{suf|en|tenuis|ous|lang1=la}}', 'root', language_dict) == [{'word':'tenuis','language':'Latin'},{'word':'ous','language':'English'}]
assert getNodesFromTemplate('{{w|Nathan Fillion}}', 'root', language_dict) == [{'word': 'Nathan Fillion', 'language': 'English'}]
assert getNodesFromTemplate('{{m|la||*brabus}}', 'root', language_dict) == [{'word': '*brabus', 'language': 'Latin'}]
assert getNodesFromTemplate('{{ja-r|もし}}', 'root', language_dict) == [{'word': 'もし', 'language': 'Japanese'}]
assert getNodesFromTemplate('{{der|en|la|vocātiō|vocātiō}}', 'root', language_dict) == [{'word': 'vocātiō', 'language': 'Latin'}]
assert getNodesFromTemplate('{{bor|sw|acx|kittāra|t=sabre}}', 'root', language_dict) == [{'word': 'kittāra', 'language': 'Omani Arabic'}]
assert getNodesFromTemplate('{{borrowed|1=la|2=grc|3=Εὐμενίδες|gloss=the gracious ones|g=f-p}}', 'root', language_dict) == [{'word': 'Εὐμενίδες', 'language': 'Ancient Greek'}]

In [ ]:
# expandTemplate('{{w|Kamil Zvelebil|K. V. Zvelebil}}')
# template = '{{inh|ja|ojp|-|sort=けさ}}'
# parseTemplates(template, session=session)
# remove_matching_parens(parseTemplates(template, session=session))
# getNodesFromTemplate(template, 'root', language_dict)
# template = '{{ja-r|もし}}'
# [d for d in data if template in d['etymology']]
# template='{{m|lij|o||the|g=m-s|pos=definite article}}'
# getNodesFromTemplate(template, 'root', language_dict), template

In [ ]:
# figure out why this broke

In [ ]:
# for desc, root, conn_type in all_connections[:]:
#     if conn_type == 'cog': continue # Ignoring cognates for now
#     if desc.startswith('{{ar-root|'):# and '=' in desc: 
#         print(desc)
#         try:
#             getWordAndLanguageFromTemplate(desc, language_dict)
#             print(getWordAndLanguageFromTemplate(desc, language_dict))
#         except:
#             print(f'Could not get info for {desc}')
#     desc_parts = splitParts(desc[2:-2])
#     desc_parts = splitParts(desc[2:-2])
#     if (desc.startswith('{{eeStart|')):
#         print(desc)

In [ ]:
# [d for d in data if '{{ar-root|س|م|ع}}' in d['etymology']]

#### Upload to Mysql

##### Start

In [ ]:
dev_conn, dev_cursor = connect('etymology_explorer_dev', user=u, password=p, host=h)

In [ ]:
len(nodeConnections)

In [ ]:
# 65s
entry_id_2_number = {d['entry_id']: d['entry_number'] for d in dev_cursor.d('SELECT entry_id, entry_number FROM entry_connections;')}
len(entry_id_2_number), entry_id_2_number[11]

In [ ]:
# 90s
unmatched_words = []
wl_2_id, next_wl_2_id = make_wl_2_id_values(dev_cursor); next_wl_2_id
# next_wl_2_id = max(wl_2_id.values()) + 1 # 7934778

In [ ]:
next_wl_2_id

In [ ]:
# 20s, Get data from section 2, data gather
entry_id_2_word = {d['entry_id']: f"{d['word']}#{d['language_name']}" for d in data}; len(entry_id_2_word)

In [ ]:
nodeConnections[1000]

In [ ]:
# 32s
# TODO need to find and match the words with "-" at the beginning or end

log_level('d')
roots = []; descs = []; table_sources = []; entry_numbers = [];

for i, (desc, root, entry_id) in enumerate(nodeConnections):
#     debug_str = f"no:{entry_id_2_number[entry_id]} {entry_id_2_word[entry_id]}, {desc['word']}&{desc['language']}, {entry_id}"
    desc_id = getOrCreateIdWithDict(desc['word'], desc['language'], dev_cursor, debug_str=None)
    
#     debug_str = f"no:{entry_id_2_number[entry_id]} {entry_id_2_word[entry_id]}, {root['word']}&{root['language']}, {entry_id}"
    root_id = getOrCreateIdWithDict(root['word'], root['language'], dev_cursor, debug_str=None)
    
    roots.append(root_id)
    descs.append(desc_id)
    table_sources.append(entry_id)
    entry_numbers.append(entry_id_2_number[entry_id])
len(unmatched_words)

In [ ]:
# get_debug_str = lambda x: f"no:{entry_id_2_number[x[0]]} {entry_id_2_word[x[0]]}, {desc['word']}&{desc['language']}, {entry_id}"
# [u*, u[0]] for u in unmatched_words]
# unmatched_words
len(unmatched_words)

In [ ]:
# del wl_2_id
# del next_wl_2_id 
# del unmatched_words

In [ ]:
# insert unmatched words
_ids,words,langs = zip(*unmatched_words)
value_dict = {'_id':_ids, 'word':words,'language_name':langs}
insert(
    dev_cursor, 
    'etymologies', 
    many=True,
    **value_dict
)

In [ ]:
dev_conn.commit()

In [ ]:
getConnectionsForID(dev_cursor, 23683)

In [ ]:
# dev_cursor.d('SELECT * FROM etymologies WHERE _id=7753025')
# dev_cursor.d('SELECT * FROM etymologies WHERE _id=297736')
# dev_cursor.d('SELECT * FROM etymologies WHERE _id=7760863')
# dev_cursor.e('SELECT table_source FROM connection_sources WHERE root=7760863 AND descendant=297736')
# dev_cursor.e('SELECT * FROM entry_etymologies WHERE entry_id=107282')
# dev_cursor.d('SELECT * FROM entry_connections WHERE entry_id=107282')
# dev_cursor.d('SELECT COUNT(*) FROM connection_sources')
# dev_cursor.d('SELECT COUNT(*) FROM connections')
# dev_cursor.d('DELETE FROM connection_sources') # ~3.5 min
# dev_cursor.d('DELETE FROM connections') # ~4 min
dev_conn.commit()

In [ ]:
[[i,t] for i,t in enumerate(table_sources) if t==254361]
roots[383842]
[w for w in wl_2_id.items() if w[1] == 7788599]

In [ ]:
len(unmatched_words)
# 700k with nothing, 
# 204381 with always transliterate some languages
# 141850 with check transliterration if needed (20s) now 150s
# 79841 with fn check transliteration and other

In [ ]:
len(roots), len(descs), len(table_sources), len(entry_numbers), list(zip(roots, descs, table_sources, entry_numbers))[0]

In [ ]:
PROBLEMS:
No _id for lexico - English, new _id = 7741531 (needs to be a prefix with '-')
No _id for graphy - English, new _id = 7741532 (needs to be a suffix with '-')
Reconstruction:Proto-Indo-Iranian/Hr̥ȷ́ipyás
What is happening with reconstructions?
Hr̥ȷ́ipyás 
"Kallyobrix"
abstraction - Middle French
वात#Etymology_1
वात#Etymology_2
ord ,  ordd, 
jen(u)ārius
fail ,  faus
fallō ,  fallis
(s)kh₃tos
K .  V . 
ǁkoaah
wikipedia : Averroes
c .
oeil ~ oil ~ uel
( mensis )  Māius
rēx ,  rēgem
quid nunc ?

Etymology problem:
The supposed name of a lost girl on the day of 2013 Bohol earthquake and used as a humorous explanation to the resulting tsunami scare in Cebu.

pneumonoultramicroscopicsilicovolcanoconiosis#English is wrong

In [ ]:
# Connection sources 1.5min
insert(
    dev_cursor, 
    'connection_sources', 
    many=True, 
    **{
        'root':roots,
        'descendant':descs,
        'table_source':table_sources,
        'entry_number':entry_numbers
    }
)

In [ ]:
dev_conn.commit()

In [ ]:
# Connection data 1 min
# make a set for roots, desc
conn_set = set(zip(roots,descs))
roots_set = [s[0] for s in conn_set]
descs_set = [s[1] for s in conn_set]

insert(
    dev_cursor, 
    'connections', 
    ignore=True,
    many=True, 
    **{
        'root':roots_set,
        'descendant':descs_set,
    }
)

Toubleshooting

In [ ]:
getConnectionsForID(dev_cursor, 89775)

In [ ]:
remove_diacritics('-ικός')

In [ ]:
word='-ικός'#'ῐκός'#'-ῐκός'
lang= 'Ancient Greek'#'English'
# _id = 9
_id = next(iter(dev_cursor.e(f'SELECT _id FROM etymologies WHERE word="{word}" AND language_name="{lang}"')),'None'); _id
# details = get_details(dev_conn, _id); details
# dev_cursor.d(f'SELECT * FROM connection_sources cs INNER JOIN etymologies e ON e._id=cs.root WHERE descendant={_id}')
# dev_cursor.d(f'SELECT * FROM connection_sources cs INNER JOIN etymologies e ON e._id=cs.descendant WHERE root={_id}')
# dev_cursor.d(f'SELECT * FROM connection_sources cs WHERE root={_id}')
# dev_cursor.d(f'SELECT * FROM entry_etymologies WHERE entry_id=623529')



In [ ]:
[d for d in descs if d==9]

In [ ]:
# dev_cursor.e('SELECT * FROM etymologies WHERE word="afraidness"') # 9
# dev_cursor.e('SELECT * FROM connections WHERE descendant=9')
# dev_cursor.e('SELECT * FROM connection_sources WHERE descendant=9')
# dev_cursor.e('SELECT * FROM connections LIMIT 1')
# dev_cursor.e('SELECT * FROM connection_sources LIMIT 1')

In [ ]:
dev_conn.commit()

In [ ]:
[d for d in data if d['word']=='frei']

In [ ]:
stage_cursor.e('SELECT * FROM connection_sources WHERE descendant=9')

In [ ]:
u = os.environ['RDS_ETY_USER']
p = os.environ['RDS_ETY_PASSWORD']
h = os.environ['RDS_ETY_HOST']
# dev_conn, dev_cursor = connect('etymology_explorer_dev', user=u, password=p, host=h)
# stage_conn, stage_cursor = connect('etymology_explorer_staging', user=u, password=p, host=h)

In [ ]:
get_

##### More troubleshooting

In [ ]:
log_level('i')

In [ ]:
getConnectionsForIndex(9)

In [ ]:
os.environ['ETY_CON_SRC_TABLE'] = 'connection_sources'
get_tree(dev_conn, _id=1504940, compression=0, details=False)

### 4. Cleanup (remove unused/weird words)

In [ ]:
a,b = wp.make_wl_2_id_values()

In [ ]:
temp = dev_cursor.d('SELECT * FROM etymologies')

In [ ]:
len(word_lang_pairs), len(temp)

In [ ]:
# Problem words
bad = [(t['word'], t['language_name']) for t in temp if '\u200e' in t['word']]
word_lang_pairs = set((w['word'], w['language_name'])  for w in temp)

In [ ]:
fixable = [b for b in bad if (b[0].replace('\u200e',''), b[1]) in word_lang_pairs]

In [ ]:
Counter(fixable).most_common()

### 5. Troubleshooting

In [ ]:
# test_cursor.e('DROP TABLE entry_etymologies')
# dev_cursor.e('SHOW CREATE TABLE entry_connections')
# test_cursor.e('CREATE TABLE `entry_etymologies` (`entry_id` int(11) NOT NULL, `wikitext` text COLLATE utf8mb4_bin, `etymology` text COLLATE utf8mb4_bin, PRIMARY KEY (`entry_id`)) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_bin')
# test_cursor.e('CREATE TABLE `entry_connections` (\n  `etymology_id` int(11) DEFAULT NULL,\n  `entry_id` int(11) DEFAULT NULL,\n  `entry_number` int(11) NOT NULL,\n  UNIQUE KEY `entry_id` (`entry_id`),\n  KEY `ety_id_to_entry` (`etymology_id`),\n  KEY `entry_number` (`entry_number`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_bin')

In [26]:
test_cursor.e('SELECT * FROM etymologies LIIMIT 100')

OperationalError: (2013, 'Lost connection to MySQL server during query')

In [4]:
log_level('d')
wp = WikiProcessor(
    '/Users/nish/development/git/wikiextractor/', 
    channel='test', 
    output_dir='output_test/',
    cache_dir=None,
    dump_file_name='test.xml',
    store_intermediates=True,
)

INFO:root:Using database: etymology_explorer_test


In [5]:
wp.process_wikidump()

INFO:root:Recreating all tables EXCEPT []
DEBUG:root:Recreating table ancestors
DEBUG:root:Recreating table common_words
DEBUG:root:Recreating table connection_sources
DEBUG:root:Recreating table connections
DEBUG:root:Recreating table entry_connections
DEBUG:root:Recreating table entry_definitions
DEBUG:root:Recreating table entry_etymologies
DEBUG:root:Recreating table entry_pos
DEBUG:root:Recreating table entry_pronunciations
DEBUG:root:Recreating table etymologies
DEBUG:root:Recreating table kin
DEBUG:root:Recreating table languages
/Users/nish/opt/anaconda3/envs/py37/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1831, "Duplicate index 'language_name_index' defined on the table 'etymology_explorer_test.languages'. This is deprecated and will be disallowed in a future release.")
  result = self._query(query)
DEBUG:root:Recreating table merged_etymologies
DEBUG:root:Recreating table permanent_errors
DEBUG:root:Recreating table progeny
DEBUG:root:Recreating table rando

Step 0/1: 

INFO:root:Replacing wikitext in the sentences...
DEBUG:root:Generating list of used wikitexts...
DEBUG:root:There are 313 to gather...
INFO:root:Generating groups of wikitexts for api calls over 4 steps...
INFO:root:Performing API calls on wikitext_groupings over 1 steps...


2220 results
Step 0/1: 

INFO:root:Replacing wikitext in the sentences...
DEBUG:root:Generating list of used wikitexts...
DEBUG:root:There are 7043 to gather...
INFO:root:Generating groups of wikitexts for api calls over 71 steps...
INFO:root:Performing API calls on wikitext_groupings over 1 steps...


...

INFO:root:Replacing wikitext in the sentences...
DEBUG:root:Generating list of used wikitexts...
DEBUG:root:There are 3163 to gather...
INFO:root:Generating groups of wikitexts for api calls over 32 steps...
INFO:root:Performing API calls on wikitext_groupings over 1 steps...


7043 results
Step 0/1: 

INFO:root:Replacing wikitext in the sentences...
INFO:root:Done converting wikitext into text...
DEBUG:root:Generating 2_id dicts for connection making...
INFO:root:Removing empty items...
INFO:root:Inserting data into mysql...
DEBUG:root:Inserting entry_etymologies into mysql...
DEBUG:root:dict_insert 2469 values: [[0, 'Borrowed from {{bor|en|ML.|dictionarium}}, f


3163 results


DEBUG:root:Inserting entry_connections into mysql...
DEBUG:root:dict_insert 4008 values: [[0, 0, 1], [1, 1, 1]]
DEBUG:root:Inserting entry_pos into mysql...
DEBUG:root:dict_insert 4011 values: [[0, 0, 'noun'], [0, 1, 'verb']]
DEBUG:root:Inserting entry_definitions into mysql...
DEBUG:root:dict_insert 6990 values: [[0, "A reference work with a list of words from o
DEBUG:root:Inserting entry_pronunciations into mysql...
DEBUG:root:dict_insert 2002 values: [[0, '(Received Pronunciation) IPA(key)/ˈdɪkʃ(ə)n(
INFO:root:Gathering wikitext parts for 2469 entries...
DEBUG:root:Gathered 4524 connections...
INFO:root:Converting connections into nodes...
INFO:root:Found 5715 unmatched words. Inserting...
DEBUG:root:number of batches: 1
DEBUG:root:in insert(), updating sql, iteration: 0
/Users/nish/opt/anaconda3/envs/py37/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1364, "Field 'common_descendant' doesn't have a default value")
  result = self._query(query)
INFO:root:Found 4203 co

In [ ]:
# parseTemplates('{{rel-top|cognates}}')
# html = expandTemplate('{{rel-top|cognates}}'); html
# text = getHtmlText(html); text
# ??parseTemplates

In [18]:
word = 'Amairgin mac Echit'
processed_wikidump = {}
# wp.load_wl_2_id_values()
# wp.load_language_dict()
# text = get_wikidump_text(word, filename='/Users/nish/development/git/wikiextractor/input/test.xml'); text
text = get_wikidump_text(word); text
# e = Extractor(0, 0, word, text); e
# text = e.transform(text); text
# text = e.wiki2text(text); text
# text = e.clean(text); text
# data = compact(text, e.title, e.reconstructed_language); data
# processed_wikidump[word] = data; processed_wikidump
# wp.get_processed_wikidump_from_single_word(word)
# wp.get_processed_text_from_single_word(word)
# ??wp.get_processed_wikidump_from_single_word
# print(g)

In [25]:
# Fix all 
#  {'pos_id': 1971, 'definition': '(punctuations) brace, curly bracket ({ })'},
#  {'pos_id': 2169, 'definition': '{|class="wikitable"'},
#  {'pos_id': 2590, 'definition': '{|class="wikitable"'},
#  {'pos_id': 3436, 'definition': '{{examples|sense=mishearing|'},
#  {'pos_id': 4000, 'definition': '{{examples|'}]
# Other similar issues?

# test_cursor.d('SELECT * FROM entry_definitions WHERE definition LIKE "%{%"')
# test_cursor.d('SELECT * FROM entry_etymologies WHERE etymology LIKE "%|%"')
# test_cursor.d('SELECT COUNT(*) FROM entry_definitions')
dev_cursor.d('SELECT * FROM entry_definitions WHERE definition LIKE "%{%"')
# test_cursor.d('SELECT * FROM entry_etymologies WHERE etymology LIKE "%|%"')
# test_cursor.d('SELECT COUNT(*) FROM entry_definitions')
# _id = 546
# _id = get_id_from_pos_id(test_conn, 2169)
# _id = get_id_word_and_language(test_conn, 'head')
# get_details(test_conn, _id)

# This is done after I rerun the processing

InterfaceError: (0, '')

In [24]:
bracesIndices = set(r for f in findMatchingBraces(text) for r in range(f[0],f[1])) # All the top-level templates
newLineIndices = [l.span()[0] for l in re.finditer(r'\n', text) if l.span()[0] not in bracesIndices]; newLineIndices # indices of new lines
parts = [text[i+1:j] for i,j in zip([-1] + newLineIndices, newLineIndices + [None])]; parts # Added [-1] to ensure beginning is captured
if len(newLineIndices) > 0:
    parts.insert(0, text[0:newLineIndices[0] ]); # Needed to get the first item in the page


In [ ]:
# test_cursor.d('SELECT * FROM entry_connections')[-1]
test_cursor.d('SELECT * FROM etymologies WHERE _id=3598')

In [ ]:
# 20s
wp.load_wl_2_id_values()
wp.load_language_dict()
en_prons_dl, en_etys_dl, en_defs_dl = wp.get_processed_text_from_single_word('banana')

In [ ]:
# for line in re.split(r'\n(?![^{]*})', text):
#     print('line', line)

In [ ]:
# def get_processed_wikidump_from_single_word(self, word):
log_level('w')
# options.acceptedNamespaces = ['w', 'wiktionary', 'wikt', 'reconstruction', 'wikipedia', 'wikispecies', 'rhymes', 'q', 'citations', 'wikisource', 'appendix']
# options.keepLinks = False
# options.expand_templates = False # templates are handled later

word='word'
processed_wikidump = {}
text = get_wikidump_text(word); text[500:700]
e = Extractor(0, 0, word, text); e
text = e.transform(text); text[500:700]
text = e.wiki2text(text); text[500:700]; text;
data = compact(e.clean(text), e.title, e.reconstructed_language); data
# processed_wikidump[word] = data
# return processed_wikidump


In [ ]:
e.clean(text); text

In [ ]:
c = e.clean(text)
[c[f[0]:f[1]] for f in findMatchingBraces(c)]

In [ ]:
for line in re.split(r'\n(?![^{]*})', e.clean(text)):
    print('line', line)

In [ ]:
# out_text = clean(text)[560:900]
out_text = 
for m in comment.finditer(text):
        spans.append((m.start(), m.end()))

In [ ]:
print(get_wiki_dump_text('word'))

In [ ]:
copy_tables(test_cursor, 'etymology_explorer_dev', 'etymology_explorer_test')

In [ ]:
# Continue

In [ ]:
u = os.environ['RDS_ETY_USER']; p = os.environ['RDS_ETY_PASSWORD']; h = os.environ['RDS_ETY_HOST']
dev_conn, dev_cursor = connect('etymology_explorer_dev', user=u, password=p, host=h)


In [ ]:
dev_cursor.e('SELECT COUNT(*) FROM connection_sources')

In [ ]:
dev_cursor.d('SELECT * FROM entry_connections ec INNER JOIN etymologies e ON e._id=ec.etymology_id WHERE entry_id=659')

In [ ]:
temp = dev_cursor.d('SELECT * FROM entry_etymologies LIMIT 10000'); temp
[s for s in temp if '{' in s['etymology'] or '}' in s['etymology']]

### 6. Create Sentiment Analysis Model data

#### Start

In [3]:
log_level('d')
wp = WikiProcessor(
    '/Users/nish/development/git/wikiextractor/', 
    channel='dev', 
    store_intermediates=True,
)
# wp = WikiProcessor('/Users/nish/development/git/wikiextractor/', channel='dev', store_intermediates=True)

In [ ]:
notify('done')

In [ ]:
# 60s?
sql_stmt = """SELECT * FROM entry_connections"""
enid_2_etid = {d['entry_id']:d['etymology_id'] for d in dev_cursor.d(sql_stmt)}

In [ ]:
# 120s
sql_stmt = """
SELECT _id, word, language_name, ee.entry_id, wikitext 
FROM entry_etymologies ee
INNER JOIN entry_connections ec ON ee.entry_id = ec.entry_id
INNER JOIN etymologies e ON e._id = ec.etymology_id
"""
wikitext_data = dev_cursor.d(sql_stmt)

In [ ]:
# Troubleshooting get_sentiment_type
from_str(', of uncertain origin -- possibly related to')
get_sentiment_type(', of uncertain origin -- possibly related to')

In [ ]:
def get_sentiment_type(text, return_id=False):
    sentiment = 'other'
    if cognate_str(text): sentiment = 'cognate'
    elif text in BRANCH_TEXTS: sentiment = 'branch'
    elif text in EQUIVALENT_TEXTS: sentiment = 'equivalent'
    elif text in RESTART_TEXTS: sentiment = 'restart'
    elif from_str(text): sentiment = 'from'
    
    if return_id:
        sentiment = stype_to_sid[sentiment]
    
    return sentiment

In [ ]:
# 80s
# wikitext_data[0]
# [s for s in set((wi['entry_id'], g['preceding_text']) for wi in wikitext_data[:10000] for g in get_wikitext_parts_dict(wi).values()) if '(' in s[1]]
# texts = set(g['preceding_text'] for wi in wikitext_data for g in get_wikitext_parts_dict(wi).values())
sentiment_data = {
    g['preceding_text']:{
        'entry_id': wi['entry_id'], 
        'sentiment_type':get_sentiment_type(g['preceding_text']) 
    } for wi in wikitext_data for g in get_wikitext_parts_dict(wi).values()
    if g['preceding_text']
}; len(sentiment_data)

In [ ]:
# train_cursor.e('DESCRIBE interwikitext_sentiment')
# train_cursor.e('DROP TABLE interwikitext_sentiment')
# train_cursor.e('CREATE TABLE `interwikitext_sentiment` (\n  `interwikitext` text COLLATE utf8mb4_bin NOT NULL,\n  `etymology_id` int(11) NOT NULL,\n  `sentiment` varchar(20) NOT NULL\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_bin')
train_cursor.e('SELECT COUNT(*) FROM interwikitext_sentiment')

In [ ]:
sentiment_dict_list = [
    {
        'interwikitext':k, 
        'etymology_id':enid_2_etid[v['entry_id']], 
        'sentiment':v['sentiment_type']
    }
    for k,v 
    in sentiment_data.items()
]; len(sentiment_dict_list), sentiment_dict_list[:5]

In [ ]:
# train_cursor.e('DESCRIBE interwikitext_sentiment')
# train_cursor.e('SELECT * FROM interwikitext_sentiment')
train_cursor.dict_insert(sentiment_dict_list, 'interwikitext_sentiment')

In [ ]:
t = '. This theory would be supported by a reading '
t= ' as a corruption of ""'
t = ', + noun of action suffix '
[w for w in wikitext_data if t in w['wikitext']]

In [ ]:
entry_id = 11276
sql_stmt = f"""
SELECT e.word, e.language_name 
FROM etymologies e 
INNER JOIN entry_connections ec ON e._id = ec.etymology_id
WHERE ec.entry_id={entry_id}"""
dev_cursor.d(sql_stmt)

dev_cursor.d(f'SELECT * FROM entry_etymologies WHERE entry_id={entry_id}')[0]['wikitext']

In [ ]:
get_sentiment_type(' : Forms of ')

In [ ]:
# texts_list = list(texts)

sentiments = [[t,get_sentiment_type(t)] for t in texts]

In [ ]:
Counter([s[1] for s in sentiments]).most_common()

In [ ]:
#Bad items
' : According to Beekes, of '
' : From the same source as '
' : of '
' : First recorded as a given name of Latvians in the end of the 19th century. From ' # period ok?
' : Modified from above. Compare '
' : According to Beekes, from the same '
' : Forms of '
' : From a poem by '
' : From the book "'
' : Possibly of '
' : Equivalent of '
' : The kanji spelling is an example of '
' : From Magadhi Prakrit, from Old-Indo-Aryan, akin to '
' : From the "műv-" stem of '
' : Usually considered a word of ' (8)
' : Related to or derived from '
' : One of the variant spellings of Nichols, from Nichol, a '
' : From the name of the local tribe, the '
' : From "*teda", from "*tendan", from '
' : Taboo of mentioning '
' : From its use in "'
' : Use of '
' : From Yueyang Lou Ji by '
' : Derived from the name "'
' : From "-er-" + '
' : From From '
' : From class 8 '
' : From unknown '
Stopped at 5

In [ ]:
t = ', ultimately from the same source as '
get_sentiment_type(t), t in texts

In [ ]:
sentiments[:50]

In [ ]:
u = os.environ['RDS_ETY_USER']; p = os.environ['RDS_ETY_PASSWORD']; h = os.environ['RDS_ETY_HOST']


In [ ]:
# sql_create_table_1 = """CREATE TABLE `interwikitext_sentiment` (`interwikitext` text COLLATE utf8mb4_bin NOT NULL, `etymology_id` int(11) NOT NULL, `sentiment_id` tinyint(1) NOT NULL) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_bin;"""
# sql_create_table_2 = """CREATE TABLE `sentiment_types` (`sentiment_id` tinyint(1) NOT NULL, `sentiment_name` text collate utf8mb4_bin NOT NULL) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_bin"""
# train_cursor.e('DROP TABLE interwikitext_sentiment')
# train_cursor.e(sql_create_table_1)
# train_cursor.e(sql_create_table_2)

In [ ]:
train_cursor.e('DESCRIBE sentiment_types')
train_cursor.e('SELECT * FROM sentiment_types')
# sentiments = [
#     [0, 'from'],
#     [1, 'cognate'],
#     [2, 'branch'],
#     [3, 'restart'],
#     [4, 'equivalent'],
#     [5, 'other'],
# ]
# train_cursor.e('INSERT INTO sentiment_types (sentiment_id, sentiment_name) VALUES (%s, %s)', values=sentiments, many=True)

In [ ]:
texts

In [ ]:
train_conn.commit()

In [ ]:
# Get all the wikitexts DONE
# Get all the intermediate texts in the wikitext DONE
# What do I want to determine? DONE
    # Cognate, From, Reset-From, Branch
# Figure out what the inputs and outputs are DONE
    # text => type
# Figure out what they would be classified as now DONE
# Make that into a dataset (columns?) 
    # interwikitext_sentiment
# Train it
# Look at the most confused examples

#### Updating sentiment based on model feedback

In [ ]:
sentiment_dict = {t['interwikitext']:t for t in train_cursor.d('SELECT * FROM interwikitext_sentiment')}; list(sentiment_dict)[:3]

In [ ]:
??get_sentiment_type

In [ ]:
text = ",accusative of "
# text = "ed spelling of "
from_str(text)
# get_sentiment_type(text)

In [ ]:
RE_FROM

In [ ]:
dev_cursor.e('SELECT * FROM entry_etymologies WHERE entry_id=4544612')

In [ ]:
log_level('d')
wp = WikiProcessor(
    '/Users/nish/development/git/wikiextractor/', 
    channel='dev', 
    store_intermediates=True,
)

In [ ]:
wikitext = 'From an unattested onomatopoeic {{etyl|hu|onomatopoeias}} stem {{af|hu|-kel|pos1=frequentative suffix|pos=verb}}. Other words with the same stem: {{l|hu|pöfög}}, {{l|hu|pöfeteg}}.'
# wikitext = 'From an {{af|hu|-kel|pos1=frequentative suffix|pos=verb}}. Other words with the same stem: {{l|hu|pöfög}}, {{l|hu|pöfeteg}}.'
wp.get_connections_from_single_wikitext(wikitext)

In [ ]:
# def connections_from_wikitext(wikitext):
wp.load_language_dict()
wtp = wp.get_wikitext_part_array([{'wikitext': wikitext, 'language_name': 'test_language', 'word': 'test_word'}])
conns, _ = wp.get_connections_from_wikitext_parts(wtp); conns
node_conns = wp.get_nodes_from_connections(conns); node_conns

In [ ]:
# Function to getWikitextsFromString -> Moved this to downstreamTasks.py
# template = 'asdf {{bababa|asdf}} asdf {{asdf|asdf}} asdf'
# def getWikitextsFromString(template_text:str)->list:
#     """Receive a string with wikitext and return the templates"""
#     return [template_text[s_i[0]:s_i[1]] for s_i in findMatchingBraces(template_text)]
getWikitextsFromString(template)

In [ ]:
# See all wikitemplates to see if there are more to ignore (loading wikitexts is 40s)
wikitext_dict = {d['entry_id']:d for d in dev_cursor.d('SELECT * from entry_etymologies')}; wikitext_dict;
# all_templates = Counter(splitParts(w[2:-2])[0] for d in wikitext_data for w in getWikitextsFromString(d[1])); all_templates
# templates_to_ignore = [];
# all_templates.most_common()

In [ ]:
# Find largest examples of wikitexts (20s)
# all_templates.most_common()[:5]
# Get a set of template type and an example of the template (the longest one)
template_examples = {}
for value in list(wikitext_dict.values()):
    for template in getWikitextsFromString(value['wikitext']):
        template_type = splitParts(template[2:-2])[0]
        if len(template_examples.setdefault(template_type, {}).setdefault('shortest', template)) > len(template):
            template_examples[template_type]['shortest'] = template
        if len(template_examples[template_type].setdefault('longest', template)) < len(template):
            template_examples[template_type]['longest'] = template
# template_examples

In [ ]:
parseTemplates('{{wikidata|Q577034|Laconic phrases}}')

In [ ]:
# Find wikitext info for some text (set wt)
# wt = '{{wikidata|Q577034|Laconic phrases}}'
for k,v in wikitext_dict.items():
    if wt in v['wikitext']:
        entry_id = k
        etymology_id = dev_cursor.e(f'SELECT etymology_id FROM entry_connections WHERE entry_id={entry_id}')[0][0]
        print(get_details(dev_conn, etymology_id))

In [ ]:
# soup.find_all('div', {'style'})
# div = soup.find_all("div", {'class':'noprint'})[0]; div
# div.decompose()
soup

In [ ]:
template_type = list(template_examples)[484]; template_type
wt = template_examples[template_type]['longest']; wt
wt = template_examples[template_type]['shortest']; wt
# wt = '{{wikisource1911Enc|Aard-wolf}}'
# wt = '{{bor|la|en|Aard-wolf}}'
html = expandTemplate(wt, session, True); html
soup = bs4.BeautifulSoup(html, features='lxml'); soup
# for div in soup.find_all("table", {'class':'metadata'}): 
#     print(div)
#     div.decompose()
# print(soup.get_text())
# text = getHtmlText(html); text
# linklessText = replaceInternalLinks(text); linklessText

In [ ]:
# Find wikitexts that dont expand into anything
for i, template_type in enumerate(list(template_examples)):
    print(f'\r{i}/{len(template_examples)}', end='')
    template_examples[template_type]['shortest_text'] = parseTemplates(template_examples[template_type]['shortest'], session=session, cache=True)
    template_examples[template_type]['longest_text'] = parseTemplates(template_examples[template_type]['longest'], session=session, cache=True)

In [ ]:
180/677
WARNING:root:unaccepted options.acceptedNamespaces: "[w" of "[w:Standard Chinese"
WARNING:root:unaccepted options.acceptedNamespaces: "[w" of "[w:Standard Chinese"
WARNING:root:unaccepted options.acceptedNamespaces: "[w" of "[w:Standard Chinese"
WARNING:root:unaccepted options.acceptedNamespaces: "[w" of "[w:Standard Chinese"
293/677
WARNING:root:unaccepted options.acceptedNamespaces: "[Appendix" of "[Appendix:Glossary#sic"
302/677
WARNING:root:unaccepted options.acceptedNamespaces: "[w" of "[w:Man'yōgana"
452/677
WARNING:root:unaccepted options.acceptedNamespaces: "[w" of "[w:Classical Chinese"
WARNING:root:unaccepted options.acceptedNamespaces: "[w" of "[w:Classical Chinese"
WARNING:root:unaccepted options.acceptedNamespaces: "[w" of "[w:Classical Chinese"
WARNING:root:unaccepted options.acceptedNamespaces: "[w" of "[w:Classical Chinese"
483/677
WARNING:root:unaccepted options.acceptedNamespaces: "[Appendix" of "[Appendix:Glossary#sic"

676/677

In [ ]:
# Get the list of templates that don't expand and those that sometimes expand
no_expand_templates = []
sometimes_expand_templates = []
never_expand_templates = []
for template_type in list(template_examples):
    se = bool(template_examples[template_type]['shortest_text'])
    le = bool(template_examples[template_type]['longest_text'])
    if not se and not le: 
        no_expand_templates.append(template_type)
    elif not se or not le: 
        sometimes_expand_templates.append(template_type)
    else:
        never_expand_templates.append(template_type)
print([len(l) for l in [no_expand_templates, sometimes_expand_templates, never_expand_templates]])


In [ ]:
# Extra no expand templates:
# wikisource1911Enc
# projectlink
# PL:1911
# wikiquote
no_expand_templates
template_examples['w']

In [ ]:
print(tailRE)
replaceInternalLinks2(out)
makeInternalLink('wikipedia:ru:Шпалерная тюрьма', '"Шпалерная тюрьма", a prison in Saint Petersburg, named according to the street where it is located')

In [ ]:
out = expandTemplate('{{w|lang=ru|Шпалерная тюрьма|"Шпалерная тюрьма", a prison in Saint Petersburg, named according to the street where it is located}}', session, True); out
# list(findMatchingBraces(out))
print(out)
replaceInternalLinks('[[out]]')

In [ ]:
dev_cursor.d('SELECT * FROM entry_connections WHERE entry_id=4610941')
get_details(dev_conn, 2071949)

In [ ]:
wikitext_data[0]

In [ ]:
'catlangname' in all_templates

In [ ]:
# A new way to expand templates using WikiExtractor.py
from WikiExtractor import Extractor, load_templates, fileinput
wt = 'banana {{w|lang=ru|Шпалерная тюрьма|"Шпалерная тюрьма", a prison in Saint Petersburg, named according to the street where it is located}} test'
e = Extractor(0, 0, 'temp', wt)

In [ ]:
e.expandTemplate('w|test')

In [ ]:
log_level('d')
# e.transform(e.text)
e.expand(e.text)

In [ ]:
# %pwd
# '/Users/nish/development/git/wikiextractor/templates'

In [ ]:
options.templates

In [ ]:
# 20s
# file = fileinput.FileInput('/Users/nish/development/git/wikiextractor/templates', openhook=fileinput.hook_compressed)
# load_templates(file)

#### Making exclude_connection_forming option for multi_parse_wikitext_sentences

In [ ]:
# wikitext_list = set(s[s_i[0]:s_i[1]] for s in sentences for s_i in findMatchingBraces(s)) 

In [ ]:
sentences = [d[0] for d in dev_cursor.e('SELECT wikitext FROM entry_etymologies LIMIT 100')]; sentences
wikitext_list = set(s[s_i[0]:s_i[1]] for s in sentences for s_i in findMatchingBraces(s)); wikitext_list
wikitext_list.add('{{etyl|test|test}}')
[[w, wikitext_is_connection_forming(w)] for w in wikitext_list if not wikitext_is_connection_forming(w)]
# wikitext_list = [w for w in wikitext_list if wikitext_is_connection_forming(w)]

In [ ]:
wt = '{{noncog|fro|livret|t=book, booklet}}'
# wt = '{{wikidata|Q577034|Laconic phrases}}'
for k,v in wikitext_dict.items():
    if wt in v['wikitext']:
        entry_id = k
        etymology_id = dev_cursor.e(f'SELECT etymology_id FROM entry_connections WHERE entry_id={entry_id}')[0][0]
        print(get_details(dev_conn, etymology_id))

# End

Get a list of sentences along with the type of connection it made

In [ ]:
# Get all the preceeding texts before wikitexts
# Really I'd like it to get the full thing and then just extract {{test}} is from {{test2}}, entry_id
# To get that data, I'll take 

In [ ]:
dev_conn, dev_cursor = connect('etymology_explorer_dev', user=u, password=p, host=h)

In [ ]:
# Temp get all word-> ids
# temp = dev_cursor.d('SELECT * FROM etymologies word'); temp
temp.sort(key=lambda x:x['word'].lower() if x['word'][:1].lower() in 'abcdefghijklmnopqrstuvwxyz' else '~'+x['word'].lower()); temp[:4]
temp_dict = {}
for t in temp:
    temp_dict.setdefault(t['language_name'], {})[t['word']] = t['_id']

list(temp_dict['English'].keys())[-10:]

In [ ]:
# sum(len(v) for v in temp_dict.values())
len(set([t.replace('\u200e', '').strip() for t in temp_dict['English'] if ' ' not in t]))

In [ ]:
with open('/Users/nish/development/projects/etymology-explorer-web/src/utils/word_dict.json', 'w') as f:
    f.write(json.dumps({'English':temp_dict['English']},separators=(',', ':')))